# Basic FISSA usage

This notebook contains a step-by-step example of how to use the function-based high-level interface to the [FISSA](https://github.com/rochefort-lab/fissa) toolbox, [fissa.run_fissa](https://fissa.readthedocs.io/en/latest/source/packages/fissa.core.html#fissa.core.run_fissa).

For more details about the methodology behind FISSA, please see our paper:

Keemink, S. W., Lowe, S. C., Pakan, J. M. P., Dylda, E., van Rossum, M. C. W., and Rochefort, N. L. FISSA: A neuropil decontamination toolbox for calcium imaging signals, *Scientific Reports*, **8**(1):3493, 2018. doi: [10.1038/s41598-018-21640-2](https://www.doi.org/10.1038/s41598-018-21640-2).

See [basic_usage_func.py](https://github.com/rochefort-lab/fissa/blob/master/examples/basic_usage_func.py) (or [basic_usage_func_windows.py](https://github.com/rochefort-lab/fissa/blob/master/examples/basic_usage_func_windows.py) for Windows users) for a short example script outside of a notebook interface.

## Import packages

First, we need to import fissa.

In [4]:
# Import the FISSA toolbox
import fissa

We also need to import some plotting dependencies which we'll make use in this notebook to display the results.

In [5]:
# For plotting our results, import numpy and matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# Fetch the colormap object for Cynthia Brewer's Paired color scheme
colors = plt.get_cmap("Paired")

## Running FISSA

With the function-based interface to FISSA, everything is handled in a single function call to [fissa.run_fissa](https://fissa.readthedocs.io/en/latest/source/packages/fissa.core.html#fissa.core.run_fissa). The function takes as its input is the raw signals, and returns the decontaminated signals.

The mandatory inputs to [fissa.run_fissa](https://fissa.readthedocs.io/en/latest/source/packages/fissa.core.html#fissa.core.run_fissa) are:

- the experiment images
- the regions of interest (ROIs) to extract

Images can be given as a path to a folder containing tiff stacks:
```python
images = "folder"
```
Each of these tiff-stacks in the folder (e.g. `"folder/trial_001.tif"`) is a trial with many frames.
Although we refer to one trial as an `image`, it is actually a video recording.

Alternatively, the image data can be given as a list of paths to tiffs:
```python
images = ["folder/trial_001.tif", "folder/trial_002.tif", "folder/trial_003.tif"]
```
or as a list of arrays which you have already loaded into memory:
```python
images = [array1, array2, array3, ...]
```

For the regions of interest (ROIs) input, you can either provide a single set of ROIs, or a set of ROIs for every image.

If the ROIs were defined using ImageJ, use ImageJ's export function to save them in a zip.
Then, provide the ROI filename.
```python
rois = "rois.zip"  # for a single set of ROIs used across all images
```
The same set of ROIs will be used for every image in `images`.

Sometimes there is motion between trials causing the alignment of the ROIs to drift.
In such a situation, you may need to use a slightly different location of the ROIs for each trial.
This can be handled by providing FISSA with a list of ROI sets — one ROI set (i.e. one ImageJ zip file) per trial.
```python
rois = ["rois1.zip", "rois2.zip", ...]  # for a unique roiset for each image
```
Please note that the ROIs defined in each ROI set must correspond to the same physical reigons across all trials, and that the order must be consistent.
That is to say, the 1st ROI listed in each ROI set must correspond to the same item appearing in each trial, etc.

In this notebook, we will demonstrate how to use FISSA with ImageJ ROI sets, saved as zip files.
However, you are not restricted to providing your ROIs to FISSA in this format.
FISSA will also accept ROIs which are arbitrarily defined by providing them as arrays (`numpy.ndarray` objects).
ROIs provided in this way can be defined either as boolean-valued masks indicating the presence of a ROI per-pixel in the image, or defined as a list of coordinates defining the boundary of the ROI.
For examples of such usage, see our [Suite2p](https://fissa.readthedocs.io/en/latest/examples/Suite2p%20example.html), [CNMF](https://fissa.readthedocs.io/en/latest/examples/cNMF%20example.html), and [SIMA](https://fissa.readthedocs.io/en/latest/examples/SIMA%20example.html) example notebooks.

As an example, we will run FISSA on a small test dataset.

The test dataset can be found and downloaded from [the examples folder of the fissa repository](https://github.com/rochefort-lab/fissa/tree/master/examples), along with the source for this example notebook.

In [7]:
import os
import glob
import sys 

import h5py
import cv2
import logging

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np


try:
    cv2.setNumThreads(0)
except:
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.paths import caiman_datadir
from caiman.source_extraction.volpy import utils
from caiman.source_extraction.volpy.volparams import volparams
from caiman.source_extraction.volpy.volpy import VOLPY
from caiman.summary_images import local_correlations_movie_offline
from caiman.summary_images import mean_image
from caiman.utils.utils import download_demo, download_model

sys.path.append(r'C:\Gomez\Analysis\2PVoltage_Imaging_Analysis_Pipeline')
from fxns.raw2tiff import *
from fxns.volpy_unpack import *
from caiman_data.fxns_caiman.plotting_fxns import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\LCG\AppData\Local\Temp\ipykernel_55172\292946207.py:24: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('load_ext autoreload')
C:\Users\LCG\AppData\Local\Temp\ipykernel_55172\292946207.py:25: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('autoreload 2')


In [8]:
def process_raw(movie_paths, mk_rawdir = False, convert = False, split = True, **kwargs):
    '''
    Process raw movies and get metadata
    '''
    exp_names = []
    tif_path = []
    exp_dir = []
    exp_metadata = dict()
    for movie in movie_paths:
        exps, metadata = raw2tiff(movie,mk_rawdir, convert, split)
        for exp in exps:
            print('Processing '+ exp)
            globals()[exp] = metadata[exp]
            globals()[exp]['file_path_ch1'] = os.path.join(metadata[exp]['exp_path'], 'analysis', 'mean_rois_ch1.csv')
            stack_path = os.path.join(metadata[exp]['exp_path'], 'tif_stack', metadata[exp]['exp_name'] + '.tif')
            tif_path += [stack_path]
            exp_dir += [os.path.split(os.path.split(stack_path)[0])[0]]
        exp_names = exp_names + exps
        exp_metadata.update(metadata)
    
    # print(exp_names)
    return tif_path, exp_names, exp_dir

In [9]:
def get_exp_paths(exp):
    stack_path = [file for file in tif_path if exp in file]
    dir_path = [file for file in exp_dir if exp in file]
    exp_name = [file for file in exp_names if exp in file]
    return stack_path, dir_path, exp_name

In [10]:
## Process Exps: Convert Raw to Tiff Stack and Collect MetaData + Get Save path 
movie_paths = [r'E:\M057\2024-02-02\MWD']
tif_path, exp_names, exp_dir = process_raw(movie_paths, mk_rawdir = False, convert = False, split = True)

Processing FOV1_MWD
Processing FOV1_Oddball
Processing FOV2_MWD
Processing FOV2_Oddball
Processing FOV3_MWD
Processing FOV3_Oddball


In [11]:
# Set motion correction parameters

opts_dict = {}
opts_dict['113Hz'] = {
    'fr': 113, # hz
    'pw_rigid': False,# flag for pw-rigid motion correction
    'max_shifts': (5, 5),  # maximum allowed rigid shift
    'gSig_filt': (3, 3), # size of filter
    'strides': (48, 48) ,  # start a new patch for pw-rigid motion correction every x pixels
    'overlaps': (24, 24),  # overlap between pathes (size of patch strides+overlaps)
    'max_deviation_rigid': 3, # maximum deviation allowed for patch with respect to rigid shifts
    'border_nan': 'copy'
}
opts_dict['147Hz'] = {
    'fr': 147, # hz
    'pw_rigid': False,# flag for pw-rigid motion correction
    'max_shifts': (5, 5),  # maximum allowed rigid shift
    'gSig_filt': (3, 3), # size of filter
    'strides': (32, 32) ,  # start a new patch for pw-rigid motion correction every x pixels
    'overlaps': (16, 16),  # overlap between pathes (size of patch strides+overlaps)
    'max_deviation_rigid': 3, # maximum deviation allowed for patch with respect to rigid shifts
    'border_nan': 'copy'
}
opts_dict['152Hz'] = {
    'fr': 153, # hz
    'pw_rigid': False,# flag for pw-rigid motion correction
    'max_shifts': (5, 5),  # maximum allowed rigid shift
    'gSig_filt': (3, 3), # size of filter
    'strides': (32, 32) ,  # start a new patch for pw-rigid motion correction every x pixels
    'overlaps': (16, 16),  # overlap between pathes (size of patch strides+overlaps)
    'max_deviation_rigid': 3, # maximum deviation allowed for patch with respect to rigid shifts
    'border_nan': 'copy'
}
opts_dict['220Hz'] = {
    'fr': 220, # hz
    'pw_rigid': False,# flag for pw-rigid motion correction
    'max_shifts': (7, 7),  # maximum allowed rigid shift
    'gSig_filt': (3, 2), # size of filter
    'strides': (32, 24) ,  # start a new patch for pw-rigid motion correction every x pixels
    'overlaps': (16, 12),  # overlap between pathes (size of patch strides+overlaps)
    'max_deviation_rigid': 3, # maximum deviation allowed for patch with respect to rigid shifts
    'border_nan': 'copy'
}
opts_dict['220Hz_awake'] = {
    'fr': 220, # hz
    'pw_rigid': False,# flag for pw-rigid motion correction
    'max_shifts': (10,10),  # maximum allowed rigid shift
    'gSig_filt': (3, 3), # size of filter
    'strides': (32, 24) ,  # start a new patch for pw-rigid motion correction every x pixels
    'overlaps': (16, 12),  # overlap between pathes (size of patch strides+overlaps)
    'max_deviation_rigid': 3, # maximum deviation allowed for patch with respect to rigid shifts
    'border_nan': 'copy'
}
opts_dict['397Hz_awake'] = {
    'fr': 397, # hz
    'pw_rigid': False,# flag for pw-rigid motion correction
    'max_shifts': (15, 15),  # maximum allowed rigid shift
    'gSig_filt': None, # size of filter
    'strides': (32, 24) ,  # start a new patch for pw-rigid motion correction every x pixels
    'overlaps': (16, 12),  # overlap between pathes (size of patch strides+overlaps)
    'max_deviation_rigid': 3, # maximum deviation allowed for patch with respect to rigid shifts
    'border_nan': 'copy'
}

In [92]:
# Set Volparams Opts
stack_path, dir_path, exps = get_exp_paths('FOV2')

opts_final = opts_dict['397Hz_awake']
opts_final['fnames'] = stack_path
opts = volparams(params_dict=opts_final)

In [93]:
# %% start a cluster for parallel processing
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

Exception: A cluster is already runnning. Terminate with dview.terminate() if you want to restart.

In [97]:
mc = MotionCorrect(stack_path, dview=dview, **opts.get_group('motion'))
mc_list = []
mc_list_ch2 = []
for exp in stack_path:
    # Load mmap file 
    mmap_file = [file for file in os.listdir(os.path.split(exp)[0]) if '.mmap' in file]
    mmap_file_ch1 = [file for file in mmap_file if 'rig' in file] # ensures ch1 is loaded
    mmap_file_ch2 = [file for file in mmap_file if 'ch2' in file] # ensures ch1 is loaded 
    mc_list += [os.path.join(os.path.split(exp)[0], mmap_file_ch1[0])]
    mc_list_ch2 += [os.path.join(os.path.split(exp)[0], mmap_file_ch2[0])]
print('mmap list ch1:')
print(mc_list)
print('mmap list ch2:')
print(mc_list_ch2)
mc.mmap_file = [mc_list[0]]
# print(f'reuse previously saved motion corrected file:{mc.mmap_file}')

mmap list ch1:
['E:\\M057\\2024-02-02\\MWD\\FOV2\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_MWD\\tif_stack\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_MWD_rig__d1_32_d2_512_d3_1_order_F_frames_360000_.mmap', 'E:\\M057\\2024-02-02\\MWD\\FOV2\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_Oddball\\tif_stack\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_Oddball_rig__d1_32_d2_512_d3_1_order_F_frames_360000_.mmap']
mmap list ch2:
['E:\\M057\\2024-02-02\\MWD\\FOV2\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_MWD\\tif_stack\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_MWD_ch2_d1_32_d2_512_d3_1_order_C_frames_360000_.mmap', 'E:\\M057\\2024-02-02\\MWD\\FOV2\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_Oddball\\tif_stack\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_Oddball_ch2_d1_32_d2_512_d3_1_order_C_frames_360000_.mmap']


In [95]:
from skimage.io import  imread, imshow, imsave

def get_x_file_paths(movie_paths, ending):
    file_paths = []
    for root, _, files in os.walk(movie_paths):
        for file in files:
            if file.endswith(ending):
                file_paths.append(os.path.join(root, file))
    return file_paths

In [96]:
mc.mmap_file[0]

'E:\\M057\\2024-02-02\\MWD\\FOV2\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_MWD\\tif_stack\\2024-02-02_M057_Drd3-cre_ASAP6-1_jRGECO1a_FOV2_MWD_rig__d1_32_d2_512_d3_1_order_F_frames_360000_.mmap'

In [107]:
# Define path to imagery and to the ROI set
Yr, dims, T = cm.load_memmap(mc.mmap_file[0])
image_stack = np.reshape(Yr.T, [T] + list(dims), order='F')[:1000] 
list_of_arrays = [image_stack[i] for i in range(image_stack.shape[0])]


roi_zip_path =get_x_file_paths(exp_dir[0], '.zip')
rois_location = roi_zip_path[0]

# Call FISSA using the function-based interface
result, raw = fissa.run_fissa(list_of_arrays, rois_location)

Extracting traces:   0%|          | 0/1000 [00:00<?, ?it/s]

ValueError: linearrings: Input operand 0 does not have enough dimensions (has 1, gufunc core with signature (i, d)->() requires 2)

The function-based interface is very straight forward, but note that you can only access the result which is returned by the function.

If you need to access the raw traces, ROI masks, or demixing matrix, you need to use the more flexible object-oriented (class based) interface using [fissa.Experiment](https://fissa.readthedocs.io/en/stable/source/packages/fissa.core.html#fissa.core.Experiment) instead.
An example of this is given in our [object-oriented example usage notebook](https://rochefort-lab.github.io/fissa/examples/Basic%20usage.html).

## Working with results

The output of ``fissa.run_fissa`` is structured as a 2-d array of 2-d arrays (it can't be a 4-d array because of trials generally don't have the same number of frames).

The results from the cell (ROI) numbered `c` and the trial (TIFF) numbered `t` are located at `result[c, t][0, :]`.

The fourth and final dimension works through frames within the TIFF file (time).

The third dimension iterates over output signals.
The 0-th entry of this is the signal which most closely corresponds to the raw signal within the ROI, and is FISSA's best guess for the decontaminated cell source signal.
The other signals are the isolated signals from contaminants such as neuropil and neighbouring cells.

Let's compare the raw signal to the separated signal for a single trial from an example ROI.

In [ ]:
# Plot sample trace

# Select the ROI and trial to plot
roi = 2
trial = 1

# Create the figure
plt.figure(figsize=(12, 6))

plt.plot(
    raw[roi, trial][0, :],
    lw=2,
    label="Raw",
    color=colors((roi * 2) % colors.N),
)
plt.plot(
    result[roi, trial][0, :],
    lw=2,
    label="Decontaminated",
    color=colors((roi * 2 + 1) % colors.N),
)

plt.title("ROI {},  Trial {}".format(roi, trial), fontsize=15)
plt.xlabel("Time (frame number)", fontsize=15)
plt.ylabel("Signal intensity (candela per unit area)", fontsize=15)
plt.grid()
plt.legend()
plt.show()

Let's next plot the traces across all ROIs and trials.

In [ ]:
# Plot all ROIs and trials

# Get the number of ROIs and trials
n_roi = result.shape[0]
n_trial = result.shape[1]

# Find the maximum signal intensities for each ROI
roi_max_raw = [
    np.max([np.max(raw[i_roi, i_trial][0]) for i_trial in range(n_trial)])
    for i_roi in range(n_roi)
]
roi_max_result = [
    np.max([np.max(result[i_roi, i_trial][0]) for i_trial in range(n_trial)])
    for i_roi in range(n_roi)
]
roi_max = np.maximum(roi_max_raw, roi_max_result)

# Plot our figure using subplot panels
plt.figure(figsize=(16, 10))
for i_roi in range(n_roi):
    for i_trial in range(n_trial):
        # Make subplot axes
        i_subplot = 1 + i_trial * n_roi + i_roi
        plt.subplot(n_trial, n_roi, i_subplot)
        # Plot the data
        plt.plot(
            raw[i_roi][i_trial][0, :],
            label="Raw",
            color=colors((i_roi * 2) % colors.N),
        )
        plt.plot(
            result[i_roi][i_trial][0, :],
            label="Decontaminated",
            color=colors((i_roi * 2 + 1) % colors.N),
        )
        # Labels and boiler plate
        plt.ylim([-0.05 * roi_max[i_roi], roi_max[i_roi] * 1.05])
        if i_roi == 0:
            plt.ylabel(
                "Trial {}\n\nSignal intensity\n(candela per unit area)".format(
                    i_trial + 1
                )
            )
        if i_trial == 0:
            plt.title("ROI {}".format(i_roi))
            plt.legend()
        if i_trial == n_trial - 1:
            plt.xlabel("Time (frame number)")

plt.show()

### Comparing ROI signal to neuropil region signals
It can be very instructive to compare the signal in the central ROI with the surrounding neuropil regions. These can be found for cell `c` and trial `t` in `raw[c, t][i, :]`, with `i=0` being the cell, and `i=1,2,3,...` indicating the surrounding regions.

Below we compare directly the raw ROI trace, the decontaminated trace, and the surrounding neuropil region traces.

In [ ]:
# Get the number of neuropil/surrounding regions.
# The raw data has the raw ROI signal in raw[:, :][0] and raw surround
# signals in the rest of the 3rd dimension.
nRegions = raw[0, 0].shape[0] - 1

# Select the ROI and trial to plot
roi = 2
trial = 1

# Create the figure
plt.figure(figsize=(12, 12))

# Plot extracted traces for each neuropil subregion
plt.subplot(2, 1, 1)
# Plot trace of raw ROI signal
plt.plot(
    raw[roi, trial][0, :],
    lw=2,
    label="Raw ROI signal",
    color=colors((roi * 2) % colors.N),
)
# Plot traces from each neuropil region
for i_neuropil in range(1, nRegions + 1):
    alpha = i_neuropil / nRegions
    plt.plot(
        raw[roi, trial][i_neuropil, :],
        lw=2,
        label="Neuropil region {}".format(i_neuropil),
        color="k",
        alpha=alpha,
    )
plt.ylim([0, 125])
plt.grid()
plt.legend()
plt.ylabel("Signal intensity (candela per unit area)", fontsize=15)
plt.title("ROI {},  Trial {}, neuropil region traces".format(roi, trial), fontsize=15)

# Plot the ROI signal
plt.subplot(2, 1, 2)
# Plot trace of raw ROI signal
plt.plot(raw[roi, trial][0, :], lw=2, label="Raw", color=colors((roi * 2) % colors.N))
# Plot decontaminated signal matched to the ROI
plt.plot(
    result[roi, trial][0, :],
    lw=2,
    label="Decontaminated",
    color=colors((roi * 2 + 1) % colors.N),
)

plt.ylim([0, 125])
plt.grid()
plt.legend()
plt.xlabel("Time (frame number)", fontsize=15)
plt.ylabel("Signal intensity (candela per unit area)", fontsize=15)
plt.title("ROI {},  Trial {}, raw and decontaminated".format(roi, trial), fontsize=15)

plt.show()

### df/f<sub>0</sub>

The default output from `fissa.run_fissa` is in the same units as the raw input (candelas per unit area).

It is often desirable to calculate the intensity of a signal relative to the baseline value, df/f<sub>0</sub>, for the traces.
`fissa.run_fissa` will do this for you provide the argument `return_deltaf=True`, and the sampling frequency of your TIFF files with `freq=sample_frequency`.
The sampling frequency must be provided because the data is smoothed in order to determine the baseline value f<sub>0</sub>.

When `return_deltaf=True`, `run_fissa` will return the df/f<sub>0</sub> output *instead* of the source signal traces scaled at the recording intensity.
If you need to access both the standard FISSA output *and* the df/f<sub>0</sub> output at the same time, you need to use the more flexible [fissa.Experiment](https://fissa.readthedocs.io/en/stable/source/packages/fissa.core.html#fissa.core.Experiment) FISSA interface instead, as described in [this example](https://fissa.readthedocs.io/en/latest/examples/Basic%20usage.html#df/f0).

In [ ]:
sample_frequency = 10  # Hz

deltaf_result, deltaf_raw = fissa.run_fissa(
    images_location, rois_location, freq=sample_frequency, return_deltaf=True
)

Note that by default, f<sub>0</sub> is determined as the minimum across all trials (all TIFFs) to ensure that results are directly comparable between trials, but you can normalise each trial individually instead if you prefer by providing the parameter `deltaf_across_trials=False`.

In [ ]:
# Plot sample trace

# Select the ROI and trial to plot
roi = 2
trial = 1

# Create the figure
plt.figure(figsize=(12, 6))

plt.plot(
    deltaf_raw[roi, trial][0, :],
    lw=2,
    label="Raw",
    color=colors((roi * 2) % colors.N),
)
plt.plot(
    deltaf_result[roi, trial][0, :],
    lw=2,
    label="Decontaminated",
    color=colors((roi * 2 + 1) % colors.N),
)

plt.title("ROI {},  Trial {}".format(roi, trial), fontsize=15)
plt.xlabel("Time (frame number)", fontsize=15)
plt.ylabel(r"$\Delta f\,/\,f_0$", fontsize=15)
plt.grid()
plt.legend()
plt.show()

Since FISSA is very good at removing contamination from the ROI signals, the minimum value on the decontaminated trace will typically be `0.`.
Consequently, we use the minimum value of the (smoothed) raw signal to provide the f<sub>0</sub> from the raw trace for both the raw and decontaminated df/f<sub>0</sub>.

We can plot the df/f<sub>0</sub> for every cell during every trial as follows.

In [ ]:
# Get the number of ROIs and trials
n_roi = result.shape[0]
n_trial = result.shape[1]

# Find the maximum signal intensities for each ROI,
# so we can scale ylim the same across subplots
roi_max = [
    np.max([np.max(result[i_roi, i_trial][0]) for i_trial in range(n_trial)])
    for i_roi in range(n_roi)
]

# Plot our figure using subplot panels
plt.figure(figsize=(16, 10))
for i_roi in range(n_roi):
    for i_trial in range(n_trial):
        # Make subplot axes
        i_subplot = 1 + i_trial * n_roi + i_roi
        plt.subplot(n_trial, n_roi, i_subplot)
        # Plot the data
        plt.plot(
            result[i_roi][i_trial][0, :],
            color=colors((i_roi * 2 + 1) % colors.N),
        )
        # Labels and boiler plate
        plt.ylim([-0.05 * roi_max[i_roi], roi_max[i_roi] * 1.05])
        if i_roi == 0:
            plt.ylabel("Trial {}\n\n".format(i_trial + 1) + r"$\Delta f\,/\,f_0$")
        if i_trial == 0:
            plt.title("ROI {}".format(i_roi))
        if i_trial == n_trial - 1:
            plt.xlabel("Time (frame number)")

plt.show()

For comparison purposes, we can also plot the df/f<sub>0</sub> for the raw data against the decontaminated signal.

In [ ]:
# Plot all ROIs and trials

# Get the number of ROIs and trials
n_roi = deltaf_result.shape[0]
n_trial = deltaf_result.shape[1]

# Find the maximum signal intensities for each ROI
roi_max_raw = [
    np.max([np.max(deltaf_raw[i_roi, i_trial][0]) for i_trial in range(n_trial)])
    for i_roi in range(n_roi)
]
roi_max_result = [
    np.max([np.max(deltaf_result[i_roi, i_trial][0]) for i_trial in range(n_trial)])
    for i_roi in range(n_roi)
]
roi_max = np.maximum(roi_max_raw, roi_max_result)

# Plot our figure using subplot panels
plt.figure(figsize=(16, 10))
for i_roi in range(n_roi):
    for i_trial in range(n_trial):
        # Make subplot axes
        i_subplot = 1 + i_trial * n_roi + i_roi
        plt.subplot(n_trial, n_roi, i_subplot)
        # Plot the data
        plt.plot(
            deltaf_raw[i_roi][i_trial][0, :],
            label="Raw",
            color=colors((i_roi * 2) % colors.N),
        )
        plt.plot(
            deltaf_result[i_roi][i_trial][0, :],
            label="Decontaminated",
            color=colors((i_roi * 2 + 1) % colors.N),
        )
        # Labels and boiler plate
        plt.ylim([-0.05 * roi_max[i_roi], roi_max[i_roi] * 1.05])
        if i_roi == 0:
            plt.ylabel("Trial {}\n\n".format(i_trial + 1) + r"$\Delta f\,/\,f_0$")
        if i_trial == 0:
            plt.title("ROI {}".format(i_roi))
            plt.legend()
        if i_trial == n_trial - 1:
            plt.xlabel("Time (frame number)")

plt.show()

## Caching

After using FISSA to decontaminate the data collected in an experiment, you will probably want to save the output for later use, so you don't have to keep re-running FISSA on the data.

To facilitate this, an option to cache the outputs is built into FISSA.
If you provide ``fissa.run_fissa`` with an identifier to the experiment being analysed in the ``folder`` argument, it will cache results into the corresponding directory.
Later, if you call ``fissa.run_fissa`` again with the same ``folder`` argument, it will load the saved results from that cache folder instead of recomputing them.

In [ ]:
# Define the folder where FISSA's outputs will be cached, so they can be
# quickly reloaded in the future without having to recompute them.
#
# This argument is optional; if it is not provided, FISSA will not save its
# results for later use.
#
# If the output directory already exists, FISSA will load the contents of
# the cache instead of recomputing it.
#
# Note: you *must* use a different folder for each experiment, otherwise
# FISSA will load the existing data instead of computing results for the
# new experiment.
#
# In this example, we will use the current datetime as the  name of the
# experiment, but you can name your experiments however you want to.
# If you want to take advantage of the caching of results, you should use
# a more descriptive name than this so you can identify the actual
# dataset that the FISSA results correspond to, and load them appropriately.

import datetime

output_folder = "fissa-example_{}".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

print(output_folder)

Let's run FISSA on this experiment again, but this time save the results to the experiment's output directory.

In [ ]:
# Run FISSA, saving to results to output_folder
result, raw = fissa.run_fissa(images_location, rois_location, folder=output_folder)

A subsequent call to ``fissa.run_fissa`` which uses the same experiment folder argument will load the cached data instead of re-running the FISSA signal separation routine from scratch.

In [ ]:
# Run FISSA, loading results from cache in output_folder
result, raw = fissa.run_fissa(images_location, rois_location, folder=output_folder)

### Exporting to MATLAB

The results can easily be exported to a MATLAB-compatible [MAT-file](https://mathworks.com/help/matlab/import_export/mat-file-versions.html) as follows.

If we provide `export_to_matfile=True` to `fissa.run_fissa`, it will export the data a matfile named `"separated.mat"` within the cache directory (the cache directory as provided with the `folder` argument).

In [ ]:
result, raw = fissa.run_fissa(
    images_location, rois_location, folder=output_folder, export_to_matfile=True
)

Alternatively, we can export to a matfile with a custom file name by setting the `export_to_matfile` argument to the target path.

In [ ]:
result, raw = fissa.run_fissa(
    images_location, rois_location, export_to_matfile="experiment_results.mat"
)

Loading the generated file (e.g. `"output_folder/separated.mat"`) in MATLAB will provide you with all of FISSA's outputs.

These are structured in the same way as the `raw` and `result` variables returned by `fissa.run_fissa`.
With the python interface, the outputs are 2d numpy.ndarrays each element of which is itself a 2d numpy.ndarrays.
Meanwhile, when the output is loaded into MATLAB the data is structured as a 2d cell-array each element of which is a 2d matrix.

Additionally, note that whilst Python indexes from 0, MATLAB indexes from 1 instead.
As a consequence of this, the results seen on Python for a given roi and trial `experiment.result[roi, trial]` correspond to the index `S.result{roi + 1, trial + 1}` on MATLAB.

Our first plot in this notebook can be replicated in MATLAB as follows:
```octave
%% Plot example traces in MATLAB
% Load FISSA output data in MATLAB
% ... either from the automatic file name within a cache
% S = load('fissa-example/separated.mat')
% ... or from a custom output path
S = load('experiment_results.mat')
% Select the second trial
% (On Python, this would be trial = 1)
trial = 2;
% Plot the result traces for each ROI
figure;
hold on;
for i_roi = 1:size(S.result, 1);
    plot(S.result{i_roi, trial}(1, :));
end
xlabel('Time (frame number)');
ylabel('Signal intensity (candela per unit area)');
grid on;
box on;
set(gca,'TickDir','out');
```

## Customisation

### Controlling verbosity

The level of verbosity of FISSA can be controlled with the `verbosity` parameter.

The default is `verbosity=1`.

If the verbosity parameter is higher, FISSA will print out more information while it is processing.
This can be helpful for debugging puproses.
The verbosity reaches its maximum at `verbosity=6`.

If `verbosity=0`, FISSA will run silently.

In [ ]:
# Call FISSA with elevated verbosity
result = fissa.run_fissa(images_location, rois_location, verbosity=2)

### Analysis parameters

FISSA has several user-definable settings, which can be set as optional arguments to `fissa.run_fissa`.

In [ ]:
# FISSA uses multiprocessing to speed up its processing.
# By default, it will spawn one worker per CPU core on your machine.
# However, if you have a lot of cores and not much memory, you many not
# be able to suport so many workers simultaneously.
# In particular, this can be problematic during the data preparation step
# in which TIFFs are loaded into memory.
# The default number of cores for the data preparation and separation steps
# can be changed as follows.
ncores_preparation = 4  # If None, uses all available cores
ncores_separation = None  # if None, uses all available cores

# By default, FISSA uses 4 subregions for the neuropil region.
# If you have very dense data with a lot of different signals per unit area,
# you may wish to increase the number of regions.
n_regions = 8

# By default, each surrounding region has the same area as the central ROI.
# i.e. expansion = 1
# However, you may wish to increase or decrease this value.
expansion = 0.75

# The degree of signal sparsity can be controlled with the alpha parameter.
alpha = 0.02

# If you change the experiment parameters, you need to change the cache directory too.
# Otherwise FISSA will try to reload the results from the previous run instead of
# computing the new results. FISSA will throw an error if you try to load data which
# was generated with different analysis parameters to the current parameters.
output_folder2 = output_folder + "_alt"

# Run FISSA with these parameters
result, raw = fissa.run_fissa(
    images_location,
    rois_location,
    output_folder2,
    nRegions=n_regions,
    expansion=expansion,
    alpha=alpha,
    ncores_preparation=ncores_preparation,
    ncores_separation=ncores_separation,
)

We can plot the new results for our example trace from before. Although we doubled the number of neuropil regions around the cell, very little has changed for this example because there were not many sources of contamination.

However, there will be more of a difference if your data has more neuropil sources per unit area within the image.

In [ ]:
n_roi = result.shape[0]
n_trial = result.shape[1]

i_roi = 3

plt.figure(figsize=(12, 6))

for i_trial in range(n_trial):
    plt.plot(result[i_roi, i_trial][0, :], label="Trial {}".format(i_trial + 1))

plt.title("ROI {}".format(i_roi), fontsize=15)
plt.xlabel("Time (Frame number)", fontsize=15)
plt.ylabel("Signal intensity (candela per unit area)", fontsize=15)
plt.grid()
plt.legend()
plt.show()

## Working with very large tiff files

By default, FISSA loads entire TIFF files into memory at once and then manipulates all ROIs within the TIFF.
This is more efficient, but can be problematic when working with very large TIFF files which are too big to be loaded into memory all at once.

If you run out of memory when running FISSA, you can try reducing the number of workers during the memory-intensive preparation step.

In [ ]:
result = fissa.run_fissa(images_location, rois_location, ncores_preparation=1)

Alternatively, you can activate FISSA's low memory mode.
In this mode, it will load and process frames one at a time.
This will run a fair bit slower than the default mode, but has a much lower memory requirement.

In [ ]:
result, raw = fissa.run_fissa(images_location, rois_location, lowmemory_mode=True)